In [315]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [268]:
pip install word2number

In [269]:
from word2number import w2n

## Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)
### Download the dataset regarding USA House Price Prediction from the following link:
https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharing
### Load the dataset and Implement 5- fold cross validation for multiple linear regression (using least square error fit).
### Steps:
### a) Divide the dataset into input features (all columns except price) and output variable (price)
### b) Scale the values of input features.
### c) Divide input and output features into five folds.
### d) Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score for each iteration using least square error fit.
### e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data and test the performance for remaining 30% data

In [368]:
df = pd.read_csv('USA_Housing.csv')
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [369]:
x = df.iloc[:,0:5]
y = df.iloc[:,5]
y = np.array(y)

In [370]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

In [373]:
def kfold(x, y, folds):
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    start = 0
    jump = int(len(x)/folds)
    end = jump
    for i in range(folds):
        x_test.append(x[start:end])
        x_train.append(np.concatenate([x[0:start], x[end:]]))
        y_test.append(y[start:end])
        y_train.append(np.concatenate([y[0:start], y[end:]]))
        start, end = end, end+jump
    return x_train, x_test, y_train, y_test

In [374]:
x_train, x_val, y_train, y_val = kfold(x_scaled, y, 5)
ans = []
for i in range(len(x_train)):
    A = x_train[i].T.dot(x_train[i])
    B = np.linalg.inv(A)
    C = B.dot(x_train[i].T)
    beta = C.dot(y_train[i])
    y_pred = x_val[i].dot(beta)
    error=y_val[i]-y_pred
    square_error=np.power(error,2)
    sum_square_error=np.sum(square_error)
    y_mean=np.mean(y_pred)
    total_variance=np.sum((y_pred-y_mean)**2)
    r2_score = 1-sum_square_error/total_variance
    ans.append((beta, r2_score))
print(ans)

[(array([1015264.04678028,  605310.05754422,  238369.32011732,
         38022.33161713,  481573.10610828]), -0.4930068812048811), (array([1007781.36699415,  606934.47748981,  270543.68001932,
         20074.53349179,  478305.19760114]), -0.5453200180202793), (array([1041608.97761308,  599780.12136812,  262781.93540361,
         20947.15150127,  452646.68047978]), -0.4981498762200103), (array([1020188.45805922,  608618.71381722,  244642.35391221,
         25204.92692526,  480455.56104238]), -0.523609680954161), (array([1038024.15512678,  590788.47767635,  261363.36006942,
         32103.41777355,  462142.97422636]), -0.5252835154791686)]


In [375]:
ans= sorted(ans, key= lambda x: x[1])
ans[-1][0]

array([1015264.04678028,  605310.05754422,  238369.32011732,
         38022.33161713,  481573.10610828])

## Q2. Concept of Validation set for Multiple Linear Regression (Gradient Descent Optimization)
### Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%).
### Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of regression coefficients for each value of learning rate after 1000 iterations.
### For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients. 

In [325]:
def cost(x, y, beta):
    total_cost = 0
    for i in range(len(x)):
        total_cost += (1/len(x))*((x[i]*beta).sum()-y[i])**2
    return total_cost

In [326]:
def gradient_descent(x, y, learning_rate, number_of_iterations):
    number_of_features = len(x.T)
    beta = np.zeros(number_of_features)
    for i in range(number_of_iterations):
        slopes = np.zeros(number_of_features)
        for j in range(len(x)):
            for k in range(number_of_features):
                slopes[k] += (1/len(x))*((x[j]*beta).sum() - y[j])*x[j][k]
        beta = beta - learning_rate*slopes
#         print(cost(x, y, beta))
    return beta

In [327]:
x = df.iloc[:,0:5]
y = df.iloc[:,5]
y = np.array(y)

In [328]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled)
x_scaled['5'] = 1
x_scaled = np.array(x_scaled)

In [329]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_scaled, y, test_size=0.3, random_state=42)

In [330]:
beta_learning = []

In [332]:
beta = gradient_descent(x_train, y_train, 0.001, 100)
beta_learning.append((beta, 0.001))
beta = gradient_descent(x_train, y_train, 0.01, 100)
beta_learning.append((beta, 0.01))
beta = gradient_descent(x_train, y_train, 0.1, 100)
beta_learning.append((beta, 0.1))
beta = gradient_descent(x_train, y_train, 1, 100)
beta_learning.append((beta, 1))

In [339]:
best_learning = []
for i in range(4):
    y_pred = x_test.dot(beta_learning[i][0])
    score = metrics.r2_score(y_test, y_pred)
    best_learning.append((score, beta_learning[i][0], beta_learning[i][1]))

In [340]:
best_learning = sorted(best_learning, key= lambda x: x[0])

In [342]:
print("Best score: ", best_learning[-1][0])
print("Best beta matrix: ", best_learning[-1][1])
print("Best learning rate: ", best_learning[-1][2])

Best score:  0.9146818498916266
Best beta matrix:  [ 230464.52520478  164159.19982569  120514.71328324    2913.62424674
  151019.35865134 1231278.63687691]
Best learning rate:  1


## Pre-processing and Multiple Linear Regression
### Download the dataset regarding Car Price Prediction from the following link:
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
### 1. Load the dataset with following column names ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"] and replace all ? values with NaN
### 2. Replace all NaN values with central tendency imputation. Drop the rows with NaN values in price column
### 3. There are 10 columns in the dataset with non-numeric values. Convert these values to numeric values using following scheme:
### (i) For “num_doors” and “num_cylinders”: convert words (number names) to figures for e.g., two to 2
### (ii) For "body_style", "drive_wheels": use dummy encoding scheme
### (iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding scheme
### (iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.
### (v) For engine_type: replace values containing string ohc to 1 else all values to 0.
### 4. Divide the dataset into input features (all columns except price) and output variable (price). Scale all input features.
### 5. Train a linear regressor on 70% of data (using inbuilt linear regression function of Python) and test its performance on remaining 30% of data.
### 6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of Python). Does it lead to any performance improvement on test set? 

In [343]:
car_price = pd.read_csv("imports-85.data")
car_price.columns = ["symboling", "normalized_losses",
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

df = car_price.copy()

In [344]:
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


In [345]:
df.replace(to_replace="?", value=np.nan, inplace=True)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


In [346]:
def f(str):
    try: 
        return w2n.word_to_num(str)
    except:
        return np.nan

In [347]:
df['num_doors'] = df['num_doors'].apply(f)
df['num_cylinders'] = df['num_cylinders'].apply(f)

In [348]:
df.dropna(subset=['price'],inplace=True)

In [349]:
for i in range(len(df.columns)):
    print(i, df.columns[i])
print(df.isnull().sum())

0 symboling
1 normalized_losses
2 make
3 fuel_type
4 aspiration
5 num_doors
6 body_style
7 drive_wheels
8 engine_location
9 wheel_base
10 length
11 width
12 height
13 curb_weight
14 engine_type
15 num_cylinders
16 engine_size
17 fuel_system
18 bore
19 stroke
20 compression_ratio
21 horsepower
22 peak_rpm
23 city_mpg
24 highway_mpg
25 price
symboling             0
normalized_losses    36
make                  0
fuel_type             0
aspiration            0
num_doors             2
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 0
dtype: int64


In [350]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df.iloc[:,[1, 5, 18, 19, 21, 22]] = imp_mean.fit_transform(df.iloc[:, [1, 5, 18, 19, 21, 22]])
df.isnull().sum().sum()

0

In [351]:
dummies = pd.get_dummies(df.body_style)
df = pd.concat([df, dummies], axis='columns')
df.drop(columns=['wagon', 'body_style'], inplace=True)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,drive_wheels,engine_location,wheel_base,length,...,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price,convertible,hardtop,hatchback,sedan
0,3,122.0,alfa-romero,gas,std,2.0,rwd,front,88.6,168.8,...,9.0,111.0,5000.0,21,27,16500,1,0,0,0
1,1,122.0,alfa-romero,gas,std,2.0,rwd,front,94.5,171.2,...,9.0,154.0,5000.0,19,26,16500,0,0,1,0
2,2,164.0,audi,gas,std,4.0,fwd,front,99.8,176.6,...,10.0,102.0,5500.0,24,30,13950,0,0,0,1
3,2,164.0,audi,gas,std,4.0,4wd,front,99.4,176.6,...,8.0,115.0,5500.0,18,22,17450,0,0,0,1
4,2,122.0,audi,gas,std,2.0,fwd,front,99.8,177.3,...,8.5,110.0,5500.0,19,25,15250,0,0,0,1


In [352]:
dummies = pd.get_dummies(df.drive_wheels)
df = pd.concat([df, dummies], axis='columns')
df.drop(columns=['rwd', 'drive_wheels'], inplace=True)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,engine_location,wheel_base,length,width,...,peak_rpm,city_mpg,highway_mpg,price,convertible,hardtop,hatchback,sedan,4wd,fwd
0,3,122.0,alfa-romero,gas,std,2.0,front,88.6,168.8,64.1,...,5000.0,21,27,16500,1,0,0,0,0,0
1,1,122.0,alfa-romero,gas,std,2.0,front,94.5,171.2,65.5,...,5000.0,19,26,16500,0,0,1,0,0,0
2,2,164.0,audi,gas,std,4.0,front,99.8,176.6,66.2,...,5500.0,24,30,13950,0,0,0,1,0,1
3,2,164.0,audi,gas,std,4.0,front,99.4,176.6,66.4,...,5500.0,18,22,17450,0,0,0,1,1,0
4,2,122.0,audi,gas,std,2.0,front,99.8,177.3,66.3,...,5500.0,19,25,15250,0,0,0,1,0,1


In [353]:
df['fuel_system'] = df['fuel_system'].apply(lambda x: 1 if "pfi" in x else 0)

In [354]:
df['engine_type'] = df['engine_type'].apply(lambda x: 1 if "ohc" in x else 0)

In [355]:
label_encoder = LabelEncoder()
df['make'] = label_encoder.fit_transform(df['make'])

In [356]:
label_encoder = LabelEncoder()
df['aspiration'] = label_encoder.fit_transform(df['aspiration'])

In [357]:
label_encoder = LabelEncoder()
df['engine_location'] = label_encoder.fit_transform(df['engine_location'])

In [358]:
label_encoder = LabelEncoder()
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])

In [359]:
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,engine_location,wheel_base,length,width,...,peak_rpm,city_mpg,highway_mpg,price,convertible,hardtop,hatchback,sedan,4wd,fwd
0,3,122.0,0,1,0,2.0,0,88.6,168.8,64.1,...,5000.0,21,27,16500,1,0,0,0,0,0
1,1,122.0,0,1,0,2.0,0,94.5,171.2,65.5,...,5000.0,19,26,16500,0,0,1,0,0,0
2,2,164.0,1,1,0,4.0,0,99.8,176.6,66.2,...,5500.0,24,30,13950,0,0,0,1,0,1
3,2,164.0,1,1,0,4.0,0,99.4,176.6,66.4,...,5500.0,18,22,17450,0,0,0,1,1,0
4,2,122.0,1,1,0,2.0,0,99.8,177.3,66.3,...,5500.0,19,25,15250,0,0,0,1,0,1


In [360]:
for i in range(len(df.columns)):
    print(i, df.columns[i])

0 symboling
1 normalized_losses
2 make
3 fuel_type
4 aspiration
5 num_doors
6 engine_location
7 wheel_base
8 length
9 width
10 height
11 curb_weight
12 engine_type
13 num_cylinders
14 engine_size
15 fuel_system
16 bore
17 stroke
18 compression_ratio
19 horsepower
20 peak_rpm
21 city_mpg
22 highway_mpg
23 price
24 convertible
25 hardtop
26 hatchback
27 sedan
28 4wd
29 fwd


In [361]:
x = df.drop(columns=['price'])
y = df.iloc[:, 23]
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

In [362]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_scaled, y, test_size=0.3, train_size=0.7)

In [363]:
alg1 = LinearRegression()

In [364]:
alg1.fit(x_train, y_train)

LinearRegression()

In [365]:
y_pred = alg1.predict(x_test)
y_pred

array([14007.59154188,  4848.12078084,  9492.61551283, 15833.99891116,
        9465.68337636,  4936.37296382, 15447.87206882, 21804.64282098,
        7702.26810378,  9011.92565987, 14067.07664422,  7916.49333916,
        6499.18431406, 15453.00587042, 10592.47917287, 14075.69154861,
        6328.6866903 ,  6470.21701377,  8820.12132644, 20943.19635503,
        9285.53950384, 20919.57382168, 16275.06005132, 27629.18259902,
       15827.54338968, 43810.79100502,  7116.97565569, 44092.85948644,
        7808.8356275 ,  6804.03429244, 10776.34005978,  6519.81481843,
       20787.17069732, 11011.05868252, 11778.29924755, -1935.52709062,
       14451.47915501,  8497.91856816, 13534.12786736,  8692.59508806,
       29632.38886592, 20228.65780483,  7436.28145341, 14457.77710853,
        8962.41399311, 16905.91430318,  8915.60255696, 31788.76087588,
        7812.34571048,  5594.49785031, 23711.24234193,  9094.94555774,
        7037.89462891,  9024.74367727, 12486.06916986, 15774.66520705,
      

In [366]:
alg1.score(x_test, y_test)

0.8023519174689698